In [ ]:
import csv
import numpy as np
import open3d as o3d
from matplotlib import pyplot as plt 
from sklearn.neighbors import KDTree
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import ipywidgets as widgets # for interactive sliders
import pickle
from timeit import timeit
from scipy.spatial import cKDTree

from itertools import product


%load_ext autoreload 
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle
from digiforest_analysis.utils.timing import Timer
timer = Timer()

In [ ]:
DATASET_DIR = "/home/ori/git/realtime-trees/single_trees/clustering_2/"
SLICE_HEIGHT = 0.5
TREE_ID = 63

In [ ]:
file_name = DATASET_DIR + "tree" + str(TREE_ID).zfill(3) + ".pkl"
with open(file_name, 'rb') as file:
    cluster = pickle.load(file)

In [ ]:
slice_height = 2.0
slice_thickness = 2.0
cloud = cluster['cloud'].point.positions.numpy()
slice = cloud[np.logical_and(cloud[:, 2] >= slice_height - slice_thickness / 2, cloud[:, 2] < slice_height + slice_thickness / 2,)][:, :2]
print(len(slice))

# timing_hough = timeit("Circle.from_cloud_hough(slice)", "from __main__ import Circle, slice", number=1000)
# print(f"Hough algo took {timing_hough:.3f} ms")
# timing_new = timeit("new_hough(slice)", "from __main__ import new_hough, slice", number=1000)
# print(f"New algo took {timing_new:.3f} ms")

with timer("OLD"):
    circ_hough = Circle.from_cloud_hough(slice)
with timer("NEW"):
    # circ_new, circles = Circle.from_cloud_ransahc(slice, min_radius=0.05, max_radius=0.5, sampling="weighted", max_points=100)
    circ_new = Circle.from_cloud_ransahc(slice, min_radius=0.05, max_radius=0.5, sampling="weighted", max_circles=500)

print(timer)
plt.scatter(slice[:, 0], slice[:, 1], s=5)
plt.gca().set_aspect('equal', adjustable='box')
# plot circle
plt.gca().add_artist(plt.Circle((circ_hough.x, circ_hough.y), circ_hough.radius, color='r', fill=False))
plt.gca().add_artist(plt.Circle((circ_new.x, circ_new.y), circ_new.radius, color='g', fill=False))
# plt.scatter(circles[:, 0], circles[:, 1], s=5, color='r')

In [ ]:
import colorsys
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
import pickle, os

%load_ext autoreload 
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle
# %matplotlib notebook

path = "/home/ori/git/digiforest_drs/trees/logs/raw"
TREE_ID = 12
tree_path = os.path.join(path, f"tree{TREE_ID:03d}.pkl")
with open(tree_path, 'rb') as file:
    tree : Tree = pickle.load(file)

def align_clouds(clusters):
    trafos_map = [c["info"]["T_sensor2map"] @ c["info"]["axis"]["transform"] for c in clusters]
    clouds_map = [c["cloud"].point.positions.numpy() @ c["info"]["T_sensor2map"][:3, :3].T + c["info"]["T_sensor2map"][:3, 3] for c in clusters]
    mean_position = np.mean([t[:3, 3] for t in trafos_map], axis=0)
    deltas = [t[:3, 3] - mean_position for t in trafos_map]
    deltas = [np.array([d[0], d[1], 0]) for d in deltas]
    return [c - d for c, d in zip(clouds_map, deltas)]



# # make 3d plot of clusters in different colors
# fig = go.Figure()
# for cluster in tree.clusters:
#     cloud = cluster['cloud'].point.positions.numpy()
#     transform = cluster['info']['T_sensor2map']
#     cloud = cloud @ transform[:3, :3].T + transform[:3, 3]
#     fig.add_trace(go.Scatter3d(x=cloud[:, 0], y=cloud[:, 1], z=cloud[:, 2], mode='markers', marker=dict(size=1)))
# fig.show()

# make a 2d plot with a slider that changes the height of the slice
clouds = [
    cluster['cloud'].point.positions.numpy() @ cluster['info']['T_sensor2map'][:3, :3].T + cluster['info']['T_sensor2map'][:3, 3]
    for cluster in tree.clusters
]
clouds_aligned = align_clouds(tree.clusters)
cloud_merged = np.concatenate(clouds, axis=0)
cloud_aligned_merged = np.concatenate(clouds_aligned, axis=0)
# cloud_aligned_merged = tree.points
hues = np.linspace(0, 1, len(clouds) + 1)[:len(clouds)]
colors = [colorsys.hsv_to_rgb(h, 1, 1) for h in hues]

slice_thickness = 0.2
# min and max width and height are 10th and 90 th percentiles
min_width, max_width = np.percentile(cloud_merged[:, 0], [2, 98])
min_height, max_height = np.percentile(cloud_merged[:, 1], [2, 98])
@widgets.interact(height=widgets.FloatSlider(min=cloud_merged[:, 2].min(), max=cloud_merged[:, 2].max(), step=0.3, value=0.0))
def update_slice_height(height):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    for a in ax:
        a.set_aspect('equal', adjustable='box')
        a.set_xlim(min_width, max_width)
        a.set_ylim(min_height, max_height)
    for cloud, color in zip(clouds, colors):
        slice = cloud[np.logical_and(cloud[:, 2] >= height - slice_thickness / 2, cloud[:, 2] < height + slice_thickness / 2,)][:, :2]
        ax[0].scatter(slice[:, 0], slice[:, 1], s=1, color=color)
    for cloud, color in zip(clouds_aligned, colors):
        slice = cloud[np.logical_and(cloud[:, 2] >= height - slice_thickness / 2, cloud[:, 2] < height + slice_thickness / 2,)][:, :2]
        ax[1].scatter(slice[:, 0], slice[:, 1], s=1, color=color)
    slice_merged = cloud_merged[np.logical_and(cloud_merged[:, 2] >= height - slice_thickness / 2, cloud_merged[:, 2] < height + slice_thickness / 2,)][:, :2]
    slice_aligned_merged = cloud_aligned_merged[np.logical_and(cloud_aligned_merged[:, 2] >= height - slice_thickness / 2, cloud_aligned_merged[:, 2] < height + slice_thickness / 2,)][:, :2]
    ax[2].scatter(slice_aligned_merged[:, 0], slice_aligned_merged[:, 1], s=1, color='k')
    # fit hough circle
    center_region = Circle(tree.axis["transform"][:3, 3], tree.axis["radius"])
    min_radius = 0.5 * tree.axis["radius"]
    max_radius = 1.5 * tree.axis["radius"]
    circ_hough = Circle.from_cloud_hough(slice_merged, min_radius=0.05, max_radius=0.5)
    circ_hough_aligned = Circle.from_cloud_hough(slice_aligned_merged, min_radius=0.05, max_radius=0.5)
    circ_ransahc = Circle.from_cloud_ransahc(slice_merged, min_radius=min_radius, max_radius=max_radius, center_region=center_region)
    circ_ransahc_aligned = Circle.from_cloud_ransahc(slice_aligned_merged, min_radius=min_radius, max_radius=max_radius, center_region=center_region)
    ax[0].add_artist(plt.Circle((circ_hough.x, circ_hough.y), circ_hough.radius, color='r', fill=False, linewidth=2))
    ax[0].add_artist(plt.Circle((circ_ransahc.x, circ_ransahc.y), circ_ransahc.radius, color='g', fill=False, linewidth=2))
    ax[2].add_artist(plt.Circle((circ_hough_aligned.x, circ_hough_aligned.y), circ_hough_aligned.radius, color='r', fill=False, linewidth=2))
    ax[2].add_artist(plt.Circle((circ_ransahc_aligned.x, circ_ransahc_aligned.y), circ_ransahc_aligned.radius, color='g', fill=False, linewidth=2))

In [ ]:
import open3d as o3d
import numpy as np
import colorsys
import pickle, os
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle

path = "/home/ori/git/digiforest_drs/trees/logs/raw"
TREE_ID = 50
tree_path = os.path.join(path, f"tree{TREE_ID:03d}.pkl")
with open(tree_path, 'rb') as file:
    tree : Tree = pickle.load(file) 

# clouds = [cluster['cloud'].transform(cluster['info']['T_sensor2map']).point.positions.numpy() for cluster in tree.clusters]
clouds = [cluster['cloud'].point.positions.numpy() @ cluster['info']['T_sensor2map'][:3, :3].T + cluster['info']['T_sensor2map'][:3, 3]for cluster in tree.clusters]
hues = np.linspace(0, 1, len(clouds) + 1) [:len(clouds)]
colors = np.concatenate([np.array([[colorsys.hls_to_rgb(h, 0.6, 1)]]*len(c)) for h, c in zip(hues, clouds)]).squeeze()
# for cluster in tree.clusters:
#     axis_trafo = cluster["info"]['T_sensor2map'] @ cluster["info"]["axis"]["transform"]
#     circle_lower = Circle(axis_trafo[:3, 3], cluster["info"]["axis"]["radius"], axis_trafo[:3, 2])
#     circle_lower = Circle(axis_trafo[:3, 3] + axis_trafo[:3, 2] * cluster)
#     verts, tris = circle_lower.generate_cone_frustum_mesh(circle_upper)
#     verts_vec = o3d.utility.Vector3dVector(verts)
#     tris_vec = o3d.utility.Vector3iVector(
#         np.concatenate((tris, np.flip(tris, axis=1)), axis=0)
#     )
#     terrain_mesh = o3d.geometry.TriangleMesh(verts_vec, tris_vec)
# raise ValueError
cloud_aligned = tree.points
o3d_cloud = o3d.t.geometry.PointCloud(np.vstack(clouds))
o3d_cloud_aligned = o3d.t.geometry.PointCloud(cloud_aligned)
o3d_cloud.point.colors = colors
o3d_cloud_aligned.point.colors = colors

# min_height = 5
# max_height = 6
# mask = np.logical_and(o3d_cloud.point.positions.numpy()[:, 2] > min_height, o3d_cloud.point.positions.numpy()[:, 2] < max_height)
# o3d_cloud = o3d_cloud.select_by_mask(mask)
# mask = np.logical_and(o3d_cloud_aligned.point.positions.numpy()[:, 2] > min_height, o3d_cloud_aligned.point.positions.numpy()[:, 2] < max_height)
# o3d_cloud_aligned = o3d_cloud_aligned.select_by_mask(mask)

aligned_cloud_flag = False

def toggle_point_cloud(vis):
    global aligned_cloud_flag
    current_view = vis.get_view_control().convert_to_pinhole_camera_parameters()
    
    if aligned_cloud_flag:
        vis.clear_geometries()
        vis.add_geometry(o3d_cloud_aligned.to_legacy())
        aligned_cloud_flag = False
        print("Showing Aligned Cloud")
    else:
        vis.clear_geometries()
        vis.add_geometry(o3d_cloud.to_legacy())
        print("Showing Non-Aligned Cloud")
        aligned_cloud_flag = True
    vis.get_view_control().convert_from_pinhole_camera_parameters(current_view, True)

visualizer = o3d.visualization.VisualizerWithKeyCallback()
visualizer.create_window()
visualizer.add_geometry(o3d_cloud.to_legacy())
visualizer.register_key_callback(ord("C"), toggle_point_cloud)
visualizer.run()
visualizer.destroy_window()

In [ ]:
import pickle, os, time
%load_ext autoreload
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle

path = "/home/ori/git/digiforest_drs/trees/logs/raw"
TREE_ID = 63
tree_path = os.path.join(path, f"tree{TREE_ID:03d}.pkl")
with open(tree_path, 'rb') as file:
    tree : Tree = pickle.load(file) 
TIME = time.perf_counter()
print(tree.reconstruct2(max_radius_deviation=100))
print(time.perf_counter() - TIME)   

# # DEBUG inside tree.reconstruct2
# from matplotlib import pyplot as plt
# import matplotlib
# import numpy as np
# import colorsys
# matplotlib.use("TKagg")
# fig, ax = plt.subplots()
# ax.set_xlabel("x")
# ax.set_ylabel("y")
# ax.set_xlim((-0.5, 0.5))
# ax.set_ylim((-0.5, 0.4))
# ax.set_title("Averaging of individual Payload Clouds")
# ax.set_aspect('equal', adjustable='box')
# hues = np.linspace(0, 1, len(ransahc_circles) + 1)[:len(ransahc_circles)]
# colors = [list(colorsys.hls_to_rgb(hue, 0.55, 0.8)) for hue in hues]
# for points, circle, color in zip(debug_slice_points, ransahc_circles, colors):
#     ax.scatter(points[:, 0], points[:, 1], c=[color]*len(points), s=2)
#     ax.add_artist(plt.Circle((circle.x, circle.y), circle.radius, color=color, fill=False))
# ax.add_artist(plt.Circle((circle.x, circle.y), circle.radius, color="g", linestyle='dashed', fill=False, linewidth=5))
# fig.show()

In [ ]:
from digiforest_analysis_ros.src.digiforest_analysis_ros.forest_analysis import TreeManager
import pickle, os

path = "/home/ori/git/digiforest_drs/trees/logs/raw_sar_exp03"
tm = TreeManager()
trees = []
for file in os.listdir(path):
    if file.endswith(".pkl"):
        with open(os.path.join(path, file), 'rb') as f:
            tree = pickle.load(f)
            trees.append(tree)
tm.trees = trees
tm.num_trees = len(trees)
# write back pickle of tm
with open(os.path.join(path, "tree_manager.pkl"), 'wb') as f:
    pickle.dump(tm, f)

# Experiments

### Data Loading

In [ ]:
import numpy as np
import open3d as o3d
import laspy
import csv, os, pickle
from scipy.interpolate import RegularGridInterpolator
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)

from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle
from digiforest_analysis_ros.forest_analysis import TreeManager
from digiforest_analysis.tasks.terrain_fitting import TerrainFitting
from digiforest_analysis.utils.distances import distance_line_to_line

datasets = {"mixed": {}, "deciduous": {}, "conifer": {}}
paths = {
    "mixed": "/home/ori/datasets/digiforest_gt_prefor_M",
    "deciduous": "/home/ori/datasets/digiforest_gt_prefor_D",
    "conifer": "/home/ori/datasets/digiforest_gt_prefor_C"
}
OURS_COLOR = "#2639DF"
TLS_COLOR = "#F07D12"
MANUAL_COLOR = "#459E11"

MIXED_MARKER = "o"
DECIDUOUS_MARKER = "x"
CONIFER_MARKER = "^"

for plot_type in datasets.keys():
    print(f"Loading {plot_type} dataset")
    tree_manager_path = os.path.join(paths[plot_type], "tree_manager.pkl")
    april_tag_pos_path = os.path.join(paths[plot_type], "april_tag_locations.csv")
    april_tag_measurement_path = os.path.join(paths[plot_type], "forester_data.csv")
    ground_cloud_path = os.path.join(paths[plot_type], "ground_cloud.las")

    # reconstructed trees
    ours_trees = []
    with open(tree_manager_path, 'rb') as file:
        tm : TreeManager = pickle.load(file)
        ours_trees = tm.trees

    # ground truth trees
    id = 0
    tls_trees = []
    for file_name in os.listdir(paths[plot_type]):
        if file_name.endswith(".csv") and "_result" in file_name:
            with open(os.path.join(paths[plot_type], file_name), 'r') as file:
                csv_reader = csv.DictReader(file)
                circle_stack = []
                for row in csv_reader:
                    circle = Circle(
                        np.array([
                            float(row["center_x"]),
                            float(row["center_y"]),
                            float(row["slice_height"])
                        ]), 
                        float(row["radius"]))
                    circle_stack.append(circle)
                
                tree = Tree(id)
                tree.reconstructed = True
                tree.circles = circle_stack
                transform = np.eye(4)
                dir_vec = circle_stack[2].center - circle_stack[1].center
                dir_vec /= np.linalg.norm(dir_vec)
                dir_vec_normal = np.array([-dir_vec[1], dir_vec[0], 0])
                dir_vec_normal /= np.linalg.norm(dir_vec_normal)
                transform[:3, 2] = dir_vec
                transform[:3, 0] = dir_vec_normal
                transform[:3, 1] = np.cross(dir_vec, dir_vec_normal)
                transform[:3, 3] = circle_stack[0].center
                tree.clusters = [{
                    "info": {
                        "T_sensor2map": np.eye(4),
                        "axis": {
                            "transform": transform,
                            "radius": circle_stack[0].radius
                        },
                        "file": file_name
                    }
                }]
                tls_trees.append(tree)
                id += 1

    # april tags
    with open(april_tag_pos_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        april_tag_positions = {int(row["tag_id"]): np.array([float(row["x"]), float(row["y"]), float(row["z"])]) for row in csv_reader}

    # manual DBHs
    dbhs = {}
    with open(april_tag_measurement_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            if row["DBH [cm]"] == "":
                continue
            dbhs[int(row["Tree"])] = float(row["DBH [cm]"].replace(",", "."))

    # tree matching
    tree_tuples = []
    for i_gt, tls_tree in enumerate(tls_trees):
        result = {"tls_tree": tls_tree,}
        gt_axis = {"transform": tls_tree.axis["transform"]}
        matches = 0
        for ours_tree in ours_trees:
            ours_axis = {"transform": ours_tree.axis["transform"]}
            dist = distance_line_to_line(gt_axis, ours_axis, clip_heights=[0,10])
            if dist < 0.5:
                result["ours_tree"] = ours_tree
        for k, v in april_tag_positions.items():
            if np.linalg.norm(v[:2] - tls_tree.axis["transform"][:2, 3]) < 0.5:
                result["april_tag"] = k 
                result["manual_dbh"] = dbhs[k]
        if "ours_tree" in result:
            tree_tuples.append(result)

    print(f"Detection Precision: {len(tree_tuples) / id * 100 :.2f} %")

    terrain_cloud = laspy.read(ground_cloud_path)
    terrain_cloud = np.vstack((terrain_cloud.x, terrain_cloud.y, terrain_cloud.z)).T
    terrain_cloud = o3d.t.geometry.PointCloud(terrain_cloud)
    terrain = TerrainFitting().process(cloud=terrain_cloud)
    terrain_interpolator = RegularGridInterpolator(
        points=(terrain[:, 0, 0], terrain[0, :, 1]),
        values=terrain[:, :, 2],
        method="linear",
        bounds_error=False,
        fill_value=None
    )
    datasets[plot_type]["tuples"] = tree_tuples
    datasets[plot_type]["terrain"] = terrain_interpolator

## Experiment 01: Accuracy

### Reconstruction

In [ ]:
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
from digiforest_analysis.utils.meshing import meshgrid_to_mesh
%load_ext autoreload
%autoreload 2
%matplotlib inline
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle

selection = None
if selection:
    tree_pair_iter = tqdm(tree_tuples[selection: selection+1])
else:
    tree_pair_iter = tqdm(tree_tuples)

manual_dbhs = []
tls_dbhs = []
ours_dbhs = []
viz_objects = []
for tree_pair in tree_pair_iter:
    ground_elevation = terrain_interpolator(tree_pair["tls_tree"].axis["transform"][:2, 3])[0]
    # manual measurement
    manual_dbhs.append(tree_pair["manual_dbh"] if "manual_dbh" in tree_pair else np.nan)
    
    # TLS measurement
    tls_tree : Tree = tree_pair["tls_tree"]
    tls_tree.compute_dbh(ground_elevation)
    tls_dbhs.append(tls_tree.dbh * 100 if tls_tree.dbh is not None else np.nan)
    
    # Ours measurement
    ours_tree : Tree = tree_pair["ours_tree"]
    if ours_tree.reconstruct3(max_radius=0.3):
        ours_tree.compute_dbh(ground_elevation)
        ours_dbhs.append(ours_tree.dbh * 100 if ours_tree.dbh is not None else np.nan)
    else:
        ours_dbhs.append(np.nan)
    
    if tls_tree.dbh is not None and ours_tree.dbh is not None:
        verts, tris = tls_tree.generate_mesh()
        mesh = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(verts), o3d.utility.Vector3iVector(tris))
        mesh.paint_uniform_color([0.2, 0.8, 0.2])
        mesh.compute_vertex_normals()
        viz_objects.append(mesh)
        
        verts, tris = ours_tree.generate_mesh()
        mesh = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(verts), o3d.utility.Vector3iVector(tris))
        mesh.paint_uniform_color([0.8, 0.2, 0.2])
        mesh.compute_vertex_normals()
        viz_objects.append(mesh)
        
        point_cloud = o3d.t.geometry.PointCloud(ours_tree.points)
        point_cloud.paint_uniform_color([0, 0, 1])
        point_cloud = point_cloud.to_legacy()
        viz_objects.append(point_cloud)

verts, tris = meshgrid_to_mesh(terrain)
mesh = o3d.geometry.TriangleMesh(
    o3d.utility.Vector3dVector(verts),
    o3d.utility.Vector3iVector(tris)
)
mesh.compute_vertex_normals()
viz_objects.append(mesh)

if not selection:
    data = []
    for i in range(len(tree_tuples)):
        tree_tuple = tree_tuples[i]
        data_dict = {
            "file_name": tree_tuple["tls_tree"].clusters[0]["info"]["file"],
            "ours_id": tree_tuple["ours_tree"].id,
            "manual_dbh": manual_dbhs[i],
            "tls_dbh": tls_dbhs[i],
            "ours_dbh": ours_dbhs[i]
        }
        if tree_tuple["ours_tree"].reconstructed:
            data_dict.update({
                "ours_centers_x": [c.center[0] for c in tree_tuple["ours_tree"].circles],
                "ours_centers_y": [c.center[1] for c in tree_tuple["ours_tree"].circles],
                "ours_centers_z": [c.center[2] for c in tree_tuple["ours_tree"].circles],
                "ours_radii": [c.radius for c in tree_tuple["ours_tree"].circles],
            })
        else: 
            data_dict.update({"ours_centers_x": [], "ours_centers_y": [], "ours_centers_z": [], "ours_diameters": []})
        if tree_tuple["tls_tree"].reconstructed:
            data_dict.update({
                "tls_centers_x": [c.center[0] for c in tree_tuple["tls_tree"].circles],
                "tls_centers_y": [c.center[1] for c in tree_tuple["tls_tree"].circles],
                "tls_centers_z": [c.center[2] for c in tree_tuple["tls_tree"].circles], 
                "tls_diameters": [c.radius for c in tree_tuple["tls_tree"].circles],
            })
        else:
            data_dict.update({"tls_centers_x": [], "tls_centers_y": [], "tls_centers_z": [], "tls_radii": []})
        data.append(data_dict)
    suffixes = {"M": "mixed", "D": "deciduous", "C": "conifer"}
    with open(f"/home/ori/git/digiforest_drs/trees/evaluation/evaluation_{suffixes[data_path[-1]]}.csv", "w+") as file:
        csv_writer = csv.DictWriter(file, fieldnames=data[0].keys())
        csv_writer.writeheader()
        csv_writer.writerows(data)

    measurements = np.stack([tls_dbhs, manual_dbhs, ours_dbhs], axis=1)
    mask = ~np.isnan(measurements).any(axis=1)
    msrmnt_triplets = measurements[mask]
    sorting = np.argsort(msrmnt_triplets[:, 1])[::-1]
    msrmnt_triplets = msrmnt_triplets[sorting]
    sorted_and_measured_inds = np.arange(len(tree_tuples))[mask][sorting]
    tls_dbhs = msrmnt_triplets[:, 0]
    manual_dbhs = msrmnt_triplets[:, 1]
    ours_dbhs = msrmnt_triplets[:, 2]
    fig_bias_std, ax_bias_std = plt.subplots(figsize=(10, 5))
    ax_bias_std.plot(range(len(manual_dbhs)), manual_dbhs, label="manual", marker='o', color=MANUAL_COLOR)
    ax_bias_std.plot(range(len(tls_dbhs)), tls_dbhs, label="TLS", marker='o', color=TLS_COLOR)
    ax_bias_std.plot(range(len(ours_dbhs)), ours_dbhs, label="Ours", marker='o', color=OURS_COLOR)
    ax_bias_std.set_xticks(range(len(sorted_and_measured_inds)), sorted_and_measured_inds)
    ax_bias_std.legend()
else:
    print(tree_tuples[selection]["ours_tree"].reconstructed)
    print(tree_tuples[selection]["tls_tree"].dbh, tree_tuples[selection]["ours_tree"].dbh) 
    o3d.visualization.draw_geometries(viz_objects)

### Evaluation

In [ ]:
def non_nan(arr):
    return arr[~np.isnan(arr)]

# Load Reconstruction Results
results = {}
evaluations = {}

for plot_type in ["mixed", "deciduous", "conifer", "all"]:
    results[plot_type] = []
    evaluations[plot_type] = []
    for i in range(3):
        if plot_type != "all":
            sub_result = {
                "file_names": [],
                "ours_ids": [],
                "manual_dbhs": [],
                "tls_trees": [],
                "ours_trees": []
            }
            sub_evaluation = {}
            # load csv
            with open(f"/home/ori/git/digiforest_drs/trees/evaluation/evaluation_{plot_type}_{i}.csv", "r") as file:
                csv_reader = csv.DictReader(file)
                for row in csv_reader:
                    sub_result["file_names"].append(row["file_name"])
                    sub_result["ours_ids"].append(int(row["ours_id"]))
                    sub_result["manual_dbhs"].append(float(row["manual_dbh"]))
                    
                    tls_tree = Tree(int(row["file_name"].split("_")[1]))
                    tls_centers_x = [float(x) for x in row["tls_centers_x"].strip("[]").split(", ") if x != ""]
                    tls_centers_y = [float(y) for y in row["tls_centers_y"].strip("[]").split(", ") if y != ""]
                    tls_centers_z = [float(z) for z in row["tls_centers_z"].strip("[]").split(", ") if z != ""]
                    tls_radii = [float(r) for r in row["tls_radii"].strip("[]").split(", ") if r != ""]
                    tls_tree.circles = [Circle(np.array([x, y, z]), r) for x, y, z, r in zip(tls_centers_x, tls_centers_y, tls_centers_z, tls_radii)]
                    tls_dbh = float(row["tls_dbh"])
                    if not np.isnan(tls_dbh):
                        tls_tree.dbh = tls_dbh
                        tls_tree.reconstructed = True
                    sub_result["tls_trees"].append(tls_tree)
                    
                    ours_tree = Tree(int(row["ours_id"]))
                    ours_centers_x = [float(x) for x in row["ours_centers_x"].strip("[]").split(", ") if x != ""]
                    ours_centers_y = [float(y) for y in row["ours_centers_y"].strip("[]").split(", ") if y != ""]
                    ours_centers_z = [float(z) for z in row["ours_centers_z"].strip("[]").split(", ") if z != ""]
                    ours_radii = [float(r) for r in row["ours_radii"].strip("[]").split(", ") if r != ""]
                    ours_tree.circles = [Circle(np.array([x, y, z]), r) for x, y, z, r in zip(ours_centers_x, ours_centers_y, ours_centers_z, ours_radii)]
                    ours_dbh = float(row["ours_dbh"])
                    if not np.isnan(ours_dbh):
                        ours_tree.dbh = ours_dbh
                        ours_tree.reconstructed = True
                    sub_result["ours_trees"].append(ours_tree)
            
            # apply global shift to fine-align ours_trees with tls_trees
            errors = []
            for tls_tree, ours_tree in zip(sub_result["tls_trees"], sub_result["ours_trees"]):
                if ours_tree.reconstructed:
                    tree_errors = []
                    for c_ours in ours_tree.circles:
                        c_tls = tls_tree.evaluate_at_height(c_ours.center[2])
                        if c_tls:
                            tree_errors.append(c_ours.center - c_tls.center)
                    errors.append(np.array(tree_errors))  
            mean_errors = np.mean(np.vstack(errors), axis=0)
            for ours_tree in sub_result["ours_trees"]:
                for circle in ours_tree.circles:
                    circle.center -= mean_errors
        elif plot_type == "all":
            sub_result = {
                "file_names": results["mixed"][i]["file_names"] + results["deciduous"][i]["file_names"] + results["conifer"][i]["file_names"],
                "ours_ids": results["mixed"][i]["ours_ids"] + results["deciduous"][i]["ours_ids"] + results["conifer"][i]["ours_ids"],
                "manual_dbhs": results["mixed"][i]["manual_dbhs"] + results["deciduous"][i]["manual_dbhs"] + results["conifer"][i]["manual_dbhs"],
                "tls_trees": results["mixed"][i]["tls_trees"] + results["deciduous"][i]["tls_trees"] + results["conifer"][i]["tls_trees"],
                "ours_trees": results["mixed"][i]["ours_trees"] + results["deciduous"][i]["ours_trees"] + results["conifer"][i]["ours_trees"]
            }
        # evaluate
        if plot_type != "all":
            ours_dbhs = np.array([tree.dbh  if tree.dbh else np.nan for tree in sub_result["ours_trees"]])
            tls_dbhs = np.array([tree.dbh  if tree.dbh else np.nan for tree in sub_result["tls_trees"]])
            manual_dbhs = np.array(sub_result["manual_dbhs"])
            sub_evaluation["dbh"] = {
                "diffs": {
                    "ours2tls": (tls_dbhs - ours_dbhs),
                    "tls2manual": (manual_dbhs - tls_dbhs),
                    "ours2manual": (manual_dbhs - ours_dbhs),
                },
                "ours" : ours_dbhs,
                "tls": tls_dbhs,
                "manual": manual_dbhs
            }
            ours_centers = []; ours_diameters = []; tls_centers = []; tls_diameters = []
            for tls_tree, ours_tree in zip(sub_result["tls_trees"], sub_result["ours_trees"]):
                oc = []; od = []; tc = []; td = []
                if ours_tree.reconstructed:
                    for c_ours in ours_tree.circles:
                        c_tls = tls_tree.evaluate_at_height(c_ours.center[2])
                        if c_tls:
                            tc.append(c_tls.center)
                            td.append(2 * c_tls.radius)
                            oc.append(c_ours.center)
                            od.append(2 * c_ours.radius)
                ours_centers.append(np.array(oc)); ours_diameters.append(np.array(od)); tls_centers.append(np.array(tc)); tls_diameters.append(np.array(td))
            sub_evaluation["stem"] = {
                "ours_centers" : ours_centers,
                "ours_diameters" : ours_diameters,
                "tls_centers" : tls_centers,
                "tls_diameters" : tls_diameters
            }
            sub_evaluation["height"] = {
                "ours_heights" : np.array([tree.circles[-1].center[2] - tree.circles[0].center[2]  if tree.reconstructed else np.nan for tree in sub_result["ours_trees"]]),
                "tls_heights" : np.array([tree.circles[-1].center[2] - tree.circles[0].center[2] for tree in sub_result["tls_trees"]])
            }
            
        elif plot_type == "all":
            sub_evaluation["dbh"] = {
                "diffs": {
                    "ours2tls" : np.hstack([evaluations["mixed"][i]["dbh"]["diffs"]["ours2tls"], evaluations["deciduous"][i]["dbh"]["diffs"]["ours2tls"], evaluations["conifer"][i]["dbh"]["diffs"]["ours2tls"]]),
                    "tls2manual" : np.hstack([evaluations["mixed"][i]["dbh"]["diffs"]["tls2manual"], evaluations["deciduous"][i]["dbh"]["diffs"]["tls2manual"], evaluations["conifer"][i]["dbh"]["diffs"]["tls2manual"]]),
                    "ours2manual" : np.hstack([evaluations["mixed"][i]["dbh"]["diffs"]["ours2manual"], evaluations["deciduous"][i]["dbh"]["diffs"]["ours2manual"], evaluations["conifer"][i]["dbh"]["diffs"]["ours2manual"]]),
                },
                "ours" : np.hstack([evaluations["mixed"][i]["dbh"]["ours"], evaluations["deciduous"][i]["dbh"]["ours"], evaluations["conifer"][i]["dbh"]["ours"]]),
                "tls" : np.hstack([evaluations["mixed"][i]["dbh"]["tls"], evaluations["deciduous"][i]["dbh"]["tls"], evaluations["conifer"][i]["dbh"]["tls"]]),
                "manual" : np.hstack([evaluations["mixed"][i]["dbh"]["manual"], evaluations["deciduous"][i]["dbh"]["manual"], evaluations["conifer"][i]["dbh"]["manual"]])
            }
            sub_evaluation["stem"] = {
                "ours_centers" : evaluations["mixed"][i]["stem"]["ours_centers"] + evaluations["deciduous"][i]["stem"]["ours_centers"] + evaluations["conifer"][i]["stem"]["ours_centers"],
                "ours_diameters" : evaluations["mixed"][i]["stem"]["ours_diameters"] + evaluations["deciduous"][i]["stem"]["ours_diameters"] + evaluations["conifer"][i]["stem"]["ours_diameters"],
                "tls_centers" : evaluations["mixed"][i]["stem"]["tls_centers"] + evaluations["deciduous"][i]["stem"]["tls_centers"] + evaluations["conifer"][i]["stem"]["tls_centers"],
                "tls_diameters" : evaluations["mixed"][i]["stem"]["tls_diameters"] + evaluations["deciduous"][i]["stem"]["tls_diameters"] + evaluations["conifer"][i]["stem"]["tls_diameters"]
            }
            sub_evaluation["height"] = {
                "ours_heights" : np.hstack([evaluations["mixed"][i]["height"]["ours_heights"], evaluations["deciduous"][i]["height"]["ours_heights"], evaluations["conifer"][i]["height"]["ours_heights"]]),
                "tls_heights" : np.hstack([evaluations["mixed"][i]["height"]["tls_heights"], evaluations["deciduous"][i]["height"]["tls_heights"], evaluations["conifer"][i]["height"]["tls_heights"]])
            }
        
        # aggregate metrics
        sub_evaluation["dbh"]["metrics"] = {
            "RMSE_ours2tls": np.sqrt(np.mean(np.square(non_nan(sub_evaluation["dbh"]["diffs"]["ours2tls"])))),
            "RMSE_ours2manual": np.sqrt(np.mean(np.square(non_nan(sub_evaluation["dbh"]["diffs"]["ours2manual"])))),
            "RMSE_tls2manual": np.sqrt(np.mean(np.square(non_nan(sub_evaluation["dbh"]["diffs"]["tls2manual"])))),
            "bias_ours2tls": np.mean(non_nan(sub_evaluation["dbh"]["diffs"]["ours2tls"])),
            "bias_ours2manual": np.mean(non_nan(sub_evaluation["dbh"]["diffs"]["ours2manual"])),
            "bias_tls2manual": np.mean(non_nan(sub_evaluation["dbh"]["diffs"]["tls2manual"])),
            "std_ours2tls": np.std(non_nan(sub_evaluation["dbh"]["diffs"]["ours2tls"])),
            "std_ours2manual": np.std(non_nan(sub_evaluation["dbh"]["diffs"]["ours2manual"])),
            "std_tls2manual": np.std(non_nan(sub_evaluation["dbh"]["diffs"]["tls2manual"])),
            "MAPE_ours2manual": np.mean(non_nan(np.abs(sub_evaluation["dbh"]["diffs"]["ours2manual"]) / sub_evaluation["dbh"]["manual"])) * 100,
            "MAPE_tls2manual": np.mean(non_nan(np.abs(sub_evaluation["dbh"]["diffs"]["tls2manual"]) / sub_evaluation["dbh"]["manual"])) * 100,
            "MAPE_ours2tls": np.mean(non_nan(np.abs(sub_evaluation["dbh"]["diffs"]["ours2tls"]) / sub_evaluation["dbh"]["tls"])) * 100,
        }
        sub_evaluation["stem"]["metrics"] = {
            "RMSE_centers": 100 * np.sqrt(np.mean(np.square(np.hstack([np.linalg.norm(ours - tls, axis=1) for ours, tls in zip(sub_evaluation["stem"]["ours_centers"], sub_evaluation["stem"]["tls_centers"]) if (ours - tls).shape[0] > 0])))),
            "RMSE_diameters": 100 * np.sqrt(np.mean(np.square(np.hstack([ours - tls for ours, tls in zip(sub_evaluation["stem"]["ours_diameters"], sub_evaluation["stem"]["tls_diameters"])])))),
            "bias_centers": 100 * np.mean(np.hstack([np.linalg.norm(ours - tls, axis=1) for ours, tls in zip(sub_evaluation["stem"]["ours_centers"], sub_evaluation["stem"]["tls_centers"]) if (ours - tls).shape[0] > 0])),
            "bias_diameters": 100 * np.mean(np.hstack([ours - tls for ours, tls in zip(sub_evaluation["stem"]["ours_diameters"], sub_evaluation["stem"]["tls_diameters"])])),
            "std_centers": 100 * np.std(np.hstack([np.linalg.norm(ours - tls, axis=1) for ours, tls in zip(sub_evaluation["stem"]["ours_centers"], sub_evaluation["stem"]["tls_centers"]) if (ours - tls).shape[0] > 0])),
            "std_diameters": 100 * np.std(np.hstack([ours - tls for ours, tls in zip(sub_evaluation["stem"]["ours_diameters"], sub_evaluation["stem"]["tls_diameters"])])),
            "MAPE_diameters": np.mean(np.abs(np.hstack([ours - tls for ours, tls in zip(sub_evaluation["stem"]["ours_diameters"], sub_evaluation["stem"]["tls_diameters"])])) / np.hstack([tls for tls in sub_evaluation["stem"]["tls_diameters"]])) * 100,
            "RMSE_rel_diameters": np.sqrt(np.mean(np.square(np.hstack([ours - tls for ours, tls in zip(sub_evaluation["stem"]["ours_diameters"], sub_evaluation["stem"]["tls_diameters"])])) / np.hstack(tls for tls in sub_evaluation["stem"]["tls_diameters"]))) * 100,
        }
        sub_evaluation["height"]["metrics"] = {
            "mean_ours": np.mean(non_nan(sub_evaluation["height"]["ours_heights"])),
            "mean_tls": np.mean(non_nan(sub_evaluation["height"]["tls_heights"])),
        }
        
        results[plot_type].append(sub_result)
        evaluations[plot_type].append(sub_evaluation)


In [ ]:
metric_names = [
    "dbh/RMSE_ours2manual",
    "dbh/RMSE_ours2tls",
    "dbh/RMSE_tls2manual",
    "",
    "dbh/RMSE_ours2manual",
    "dbh/bias_ours2manual",
    "dbh/std_ours2manual",
    "",
    "dbh/bias_ours2tls",
    "dbh/bias_ours2manual",
    "dbh/bias_tls2manual",
    "",
    "dbh/MAPE_ours2manual",
    "dbh/MAPE_tls2manual",
    "dbh/MAPE_ours2tls",
    "-",
    "stem/RMSE_centers",
    "stem/bias_centers",
    "stem/std_centers",
    "",
    "stem/RMSE_diameters",
    "stem/bias_diameters",
    "stem/std_diameters",
    "-",
    "height/mean_ours",
    "height/mean_tls",
]

print(f"\033[1mmetric_name         |     conifer       mixed   deciduous |        all\033[0m")
print(f"----------------------------------------------------------------------")
for metric_name in metric_names:
    if metric_name == "":
        print()
        continue
    if metric_name == "-":
        print("----------------------------------------------------------------------")
        continue
    metric_first, metric_second = metric_name.split("/")
    mean_metric_conifer = np.mean([e[metric_first]["metrics"][metric_second] for e in evaluations["conifer"]])
    mean_metric_mixed = np.mean([e[metric_first]["metrics"][metric_second] for e in evaluations["mixed"]])
    mean_metric_deciduous = np.mean([e[metric_first]["metrics"][metric_second] for e in evaluations["deciduous"]])
    mean_metric_all = np.mean([e[metric_first]["metrics"][metric_second] for e in evaluations["all"]])
    print(f"{metric_name:<20}|{mean_metric_conifer:>12.2f}{mean_metric_mixed:>12.2f}{mean_metric_deciduous:>12.2f} | {mean_metric_all:>10.2f}")
 

In [ ]:
 8.36        6.30        3.30 |       6.12
17.16       15.22        5.94 |      10.22

### Plotting

In [ ]:

import matplotlib
matplotlib.rc('text', usetex=True)
# tls manual ours

plot_config = {
    "mixed": {
        "marker": MIXED_MARKER,
        "marker_size": 20,
        "title": "Mixed Plot",
    },
    "deciduous": {
        "marker": DECIDUOUS_MARKER,
        "marker_size": 30,
        "title":"Deciduous Trees",
        },
    "conifer": {
        "marker": CONIFER_MARKER,
        "marker_size": 15,
        "title": "Conifer Trees",
    },
    "all": {
        "title":"All Trees",
    }
}

non_nan_row = lambda arr: arr[~np.isnan(arr).any(axis=1)]
data_points = {
    "mixed" : {
        "ours2tls": non_nan_row(np.vstack([evaluations["mixed"][i]["dbh"]["ours"], evaluations["mixed"][i]["dbh"]["tls"]]).T),
        "ours2manual": non_nan_row(np.vstack([evaluations["mixed"][i]["dbh"]["ours"], evaluations["mixed"][i]["dbh"]["manual"]]).T)
    },
    "deciduous" : {
        "ours2tls": non_nan_row(np.vstack([evaluations["deciduous"][i]["dbh"]["ours"], evaluations["deciduous"][i]["dbh"]["tls"]]).T),
        "ours2manual": non_nan_row(np.vstack([evaluations["deciduous"][i]["dbh"]["ours"], evaluations["deciduous"][i]["dbh"]["manual"]]).T)
    },
    "conifer" : {
        "ours2tls": non_nan_row(np.vstack([evaluations["conifer"][i]["dbh"]["ours"], evaluations["conifer"][i]["dbh"]["tls"]]).T),
        "ours2manual": non_nan_row(np.vstack([evaluations["conifer"][i]["dbh"]["ours"], evaluations["conifer"][i]["dbh"]["manual"]]).T)
    }
}
data_points["mixed"]["min_radius"] = data_points["mixed"]["ours2tls"].min()
data_points["mixed"]["max_radius"] = data_points["mixed"]["ours2tls"].max()
data_points["deciduous"]["min_radius"] = data_points["deciduous"]["ours2tls"].min()
data_points["deciduous"]["max_radius"] = data_points["deciduous"]["ours2tls"].max()
data_points["conifer"]["min_radius"] = data_points["conifer"]["ours2tls"].min()
data_points["conifer"]["max_radius"] = data_points["conifer"]["ours2tls"].max()
data_points["all"] = {
    "min_radius": np.min([data_points["mixed"]["min_radius"], data_points["deciduous"]["min_radius"], data_points["conifer"]["min_radius"]]),
    "max_radius": np.max([data_points["mixed"]["max_radius"], data_points["deciduous"]["max_radius"], data_points["conifer"]["max_radius"]]),
}

index = 2
for plot_type in ["mixed", "deciduous", "conifer", "all"]:
    fig_bias_std, ax_bias_std = plt.subplots(1, 1, figsize=(3.3, 3.3))
    fig_bias_std.tight_layout()
    min_radius = data_points[plot_type]["min_radius"]
    max_radius = data_points[plot_type]["max_radius"]
    ax_bias_std.plot([min_radius, max_radius], [min_radius, max_radius], color="#6E6E6E", linestyle='dashed', zorder=0)
    if plot_type == "all":
        ax_bias_std.scatter(data_points["conifer"]["ours2tls"][:, 0], data_points["conifer"]["ours2tls"][:, 1], marker=plot_config["conifer"]["marker"], s=plot_config["conifer"]["marker_size"], color=TLS_COLOR, zorder=1)
        ax_bias_std.scatter(data_points["conifer"]["ours2manual"][:, 0], data_points["conifer"]["ours2manual"][:, 1], marker=plot_config["conifer"]["marker"], s=plot_config["conifer"]["marker_size"], color=MANUAL_COLOR, zorder=1)
        ax_bias_std.scatter(data_points["mixed"]["ours2tls"][:, 0], data_points["mixed"]["ours2tls"][:, 1], marker=plot_config["mixed"]["marker"], s=plot_config["mixed"]["marker_size"], color=TLS_COLOR, zorder=1)
        ax_bias_std.scatter(data_points["mixed"]["ours2manual"][:, 0], data_points["mixed"]["ours2manual"][:, 1], marker=plot_config["mixed"]["marker"], s=plot_config["mixed"]["marker_size"], color=MANUAL_COLOR, zorder=1)
        ax_bias_std.scatter(data_points["deciduous"]["ours2tls"][:, 0], data_points["deciduous"]["ours2tls"][:, 1], marker=plot_config["deciduous"]["marker"], s=plot_config["deciduous"]["marker_size"], color=TLS_COLOR, zorder=1)
        ax_bias_std.scatter(data_points["deciduous"]["ours2manual"][:, 0], data_points["deciduous"]["ours2manual"][:, 1], marker=plot_config["deciduous"]["marker"], s=plot_config["deciduous"]["marker_size"], color=MANUAL_COLOR, zorder=1)
        legend_elements = [
            matplotlib.lines.Line2D([0], [0], marker=MIXED_MARKER, color='w', markerfacecolor='k', markersize=10, label='Mixed'),
            matplotlib.lines.Line2D([0], [0], marker=DECIDUOUS_MARKER, color='k', markerfacecolor='k', linestyle='', markersize=7, label='Deciduous'),
            matplotlib.lines.Line2D([0], [0], marker=CONIFER_MARKER, color='w', markerfacecolor='k', markersize=10, label='Conifer'),
            matplotlib.patches.Patch(color=TLS_COLOR, label='wrt. TLS'),
            matplotlib.patches.Patch(color=MANUAL_COLOR, label='wrt. Manual'),
        ]
        ax_bias_std.legend(handles=legend_elements, loc='upper left')
    else:
        ax_bias_std.scatter(data_points[plot_type]["ours2tls"][:, 0], data_points[plot_type]["ours2tls"][:, 1], marker=plot_config[plot_type]["marker"], s=plot_config[plot_type]["marker_size"], color=TLS_COLOR, zorder=1)
        ax_bias_std.scatter(data_points[plot_type]["ours2manual"][:, 0], data_points[plot_type]["ours2manual"][:, 1], marker=plot_config[plot_type]["marker"], s=plot_config[plot_type]["marker_size"], color=MANUAL_COLOR, zorder=1)
    # ax.set_xlabel("Our DBH [cm]")
    # ax.set_ylabel("Reference DBH [cm]")
    ax_bias_std.set_title(plot_config[plot_type]["title"])
    fig_bias_std.savefig(f"/home/ori/Documents/Leonard/drs-docs-current/2024-iros-realtime-trees/pics/dbh_scatter_{plot_type}.pdf", bbox_inches='tight')


In [ ]:
o3d.visualization.draw_geometries(viz_objects)

In [ ]:
import ipywidgets as widgets
%matplotlib inline

@widgets.interact(idx=widgets.IntSlider(min=0, max=len(centers_ours)-1, step=1, value=0))
def update_slice_height(idx):
    fig, ax = plt.subplots(3, 1, figsize=(15, 8))
    print(f"Corr X: {correlation_x[idx]:.3f}, Corr Y: {correlation_y[idx]:.3f}, Corr Radius: {correlation_radius[idx]:.3f}")
    print(f"Mean Error Dist: {mean_errors_dist[idx]*100:.3f} cm, Mean Error Radius: {mean_errors_radius[idx]*100:.3f} cm")
    ax[0].plot(range(len(centers_ours[idx])), [c[0] for c in centers_ours[idx]], label="Ours", marker='o', color='r')
    ax[0].plot(range(len(centers_gt[idx])), [c[0] for c in centers_gt[idx]], label="GT", marker='o', color='g')
    ax[0].set_title("X")
    ax[0].legend()
    ax[1].plot(range(len(centers_ours[idx])), [c[1] for c in centers_ours[idx]], label="Ours", marker='o', color='r')
    ax[1].plot(range(len(centers_gt[idx])), [c[1] for c in centers_gt[idx]], label="GT", marker='o', color='g')
    ax[1].set_title("Y")
    ax[1].legend()
    ax[2].plot(range(len(radii_ours[idx])), radii_ours[idx], label="Ours", marker='o', color='r')
    ax[2].plot(range(len(radii_gt[idx])), radii_gt[idx], label="GT", marker='o', color='g')
    ax[2].set_title("Radius")
    ax[2].legend()
    # fig.show()
 

In [ ]:
o3d.visualization.draw_geometries(viz_objects)

## Experiment 02: Global Consistency

## Experiment 03: Ablations

### Coverage Angle

In [ ]:
from tqdm.auto import tqdm
coverage_ablation_results = []

for dataset in datasets.values():
    tree_tuples = dataset["tuples"]
    terrain_interpolator = dataset["terrain"]
    for tree_tuple in tqdm(tree_tuples):
        ground_elevation = terrain_interpolator(ours_tree.axis["transform"][:2, 3])[0]
        tree_tuple["tls_tree"].compute_dbh(ground_elevation)
        sub_result = {"tls_dbh" : tree_tuple["tls_tree"].dbh * 100 if tree_tuple["tls_tree"].dbh is not None else np.nan, "manual_dbh" : tree_tuple["manual_dbh"] if "manual_dbh" in tree_tuple else np.nan, "ours_dbh": []}
        ours_tree : Tree = tree_tuple["ours_tree"]
        ours_tree_clusters = ours_tree.clusters
        cluster_coverage_angles = np.array([(c["info"]["coverage"]["angle_from"], c["info"]["coverage"]["angle_to"]) for c in ours_tree_clusters])
        sort_indices = np.argsort([angle_to - angle_from for angle_from, angle_to in cluster_coverage_angles])
        for i_sort in range(1, len(sort_indices)):
            ours_tree.clusters = [ours_tree_clusters[i_cluster] for i_cluster in sort_indices[:i_sort]]
            coverage_angle = tm.compute_angle_coverage(cluster_coverage_angles[sort_indices[:i_sort]])
            ours_tree.dbh = None
            reco_success = ours_tree.reconstruct3()
            ours_tree.compute_dbh(ground_elevation)
            if reco_success and ours_tree.dbh is not None:
                reference = tree_tuple["manual_dbh"] if "manual_dbh" in tree_tuple else tree_tuple["tls_tree"].dbh * 100 if tree_tuple["tls_tree"].dbh is not None else np.nan
                if not np.isnan(reference) and abs(ours_tree.dbh * 100 - reference) > 10:
                    sub_result["ours_dbh"].append({"coverage_angle" : coverage_angle, "dbh" : np.nan})
                else:
                    sub_result["ours_dbh"].append({"coverage_angle" : coverage_angle, "dbh" : ours_tree.dbh * 100})
            else:
                sub_result["ours_dbh"].append({"coverage_angle" : coverage_angle, "dbh" : np.nan})
                
        ours_tree.clusters = ours_tree_clusters
        coverage_ablation_results.append(sub_result)

In [ ]:
from matplotlib import pyplot as plt
from scipy.interpolate import make_interp_spline
import matplotlib
matplotlib.rc('text', usetex=True)

# 300 rep

def smoothify(x, *y):
    xnew = np.linspace(x.min(), x.max(), 1000)
    y_new = []
    for y_i in y:
        spl = make_interp_spline(x, y_i, k=3)
        y_new.append(spl(xnew))
    return xnew, *y_new
    

coverage_angles = []
errors_tls = []
errors_manual = []
for result in coverage_ablation_results:
    coverage_angles.extend([np.rad2deg(r["coverage_angle"]) for r in result["ours_dbh"]])
    errors_tls.extend([r["dbh"] - result["tls_dbh"] for r in result["ours_dbh"]])
    errors_manual.extend([r["dbh"] - result["manual_dbh"] for r in result["ours_dbh"]])
coverage_angles = np.array(coverage_angles)
errors_tls = np.array(errors_tls)
mask_tls = ~np.isnan(np.vstack([coverage_angles, errors_tls])).any(axis=0)
errors_tls = errors_tls[mask_tls]
coverage_angles_tls = coverage_angles[mask_tls]
errors_manual = np.array(errors_manual)
mask_manual = ~np.isnan(np.vstack([coverage_angles, errors_manual])).any(axis=0)    
errors_manual = errors_manual[mask_manual]
coverage_angles_manual = coverage_angles[mask_manual]


means_tls = []; stds_tls = []; means_manual = []; stds_manual = []; rmses_tls = []; rmses_manual = []
num_buckets = 15
bucket_boundaries = np.linspace(0, 360, num_buckets + 1, endpoint=True)
bucket_centers = (bucket_boundaries[1:] + bucket_boundaries[:-1]) / 2
bucket_mask_tls = np.zeros(len(bucket_centers), dtype=bool)
bucket_mask_manual = np.zeros(len(bucket_centers), dtype=bool)
for i in range(len(bucket_boundaries) - 1):
    # tls
    mask_tls = np.logical_and(coverage_angles_tls > bucket_boundaries[i], coverage_angles_tls <= bucket_boundaries[i + 1])
    if mask_tls.sum() > 3:
        bucket_tls = errors_tls[mask_tls]
        means_tls.append(np.mean(bucket_tls))
        rmses_tls.append(np.sqrt(np.mean(bucket_tls ** 2)))
        stds_tls.append(np.std(bucket_tls))
        bucket_mask_tls[i] = True
    
    # manual
    mask_manual = np.logical_and(coverage_angles_manual > bucket_boundaries[i], coverage_angles_manual <= bucket_boundaries[i + 1])
    if mask_manual.sum() > 3:
        bucket_manual = errors_manual[mask_manual]
        means_manual.append(np.mean(bucket_manual))
        rmses_manual.append(np.sqrt(np.mean(bucket_manual ** 2)))
        stds_manual.append(np.std(bucket_manual))
        bucket_mask_manual[i] = True
        
bucket_centers_tls = bucket_centers[bucket_mask_tls]
bucket_centers_manual = bucket_centers[bucket_mask_manual]
means_tls = np.array(means_tls); stds_tls = np.array(stds_tls); means_manual = np.array(means_manual); stds_manual = np.array(stds_manual); rmses_tls = np.array(rmses_tls); rmses_manual = np.array(rmses_manual)


fig_rmse, ax_rmse = plt.subplots(1, 1, figsize=(7, 3))
# tls
ax_rmse.scatter(bucket_centers_tls, rmses_tls, label="TLS", marker="s", s=20, color=TLS_COLOR)
c = np.linalg.lstsq(np.vstack([bucket_centers_tls, np.ones(len(bucket_centers_tls))]).T, rmses_tls, rcond=None)[0] # fit lsq line to rmses
ax_rmse.plot([0, 360], [c[1], c[0] * 360 + c[1]], color=TLS_COLOR, linestyle='dashed')
# ax_rmse.scatter(coverage_angles_tls, errors_tls, color=TLS_COLOR, s=1)
# manual
ax_rmse.scatter(bucket_centers_manual, rmses_manual, label="Manual", marker="s", s=20, color=MANUAL_COLOR)
c = np.linalg.lstsq(np.vstack([bucket_centers_manual, np.ones(len(bucket_centers_manual))]).T, rmses_manual, rcond=None)[0] # fit lsq line to rmses
ax_rmse.plot([0, 360], [c[1], c[0] * 360 + c[1]], color=MANUAL_COLOR, linestyle='dashed')
# ax_rmse.scatter(coverage_angles_manual, errors_manual, color=MANUAL_COLOR, s=1)
ax_rmse.set_xlabel("Coverage Angle [deg]")
ax_rmse.set_ylabel("RMSE [cm]")
ax_rmse.legend()
fig_rmse.savefig("/home/ori/Documents/Leonard/drs-docs-current/2024-iros-realtime-trees/pics/coverage_ablation_rmse.pdf", bbox_inches='tight')


fig_bias_std, ax_bias_std = plt.subplots(1, 1, figsize=(7, 3))
ax_bias_std.plot([0, 360], [0, 0], color="#6E6E6E", linestyle='dashed', zorder=0)
# tls
# bucket_centers_tls, means_tls, stds_tls = smoothify(bucket_centers_tls, means_tls, stds_tls)
ax_bias_std.plot(bucket_centers_tls, means_tls, label="TLS", color=TLS_COLOR)
ax_bias_std.fill_between(bucket_centers_tls, means_tls - stds_tls, means_tls + stds_tls, color=TLS_COLOR, alpha=0.3)
# manual
# bucket_centers_manual, means_manual, stds_manual = smoothify(bucket_centers_manual, means_manual, stds_manual)
ax_bias_std.plot(bucket_centers_manual, means_manual, label="Manual", color=MANUAL_COLOR)
ax_bias_std.fill_between(bucket_centers_manual, means_manual - stds_manual, means_manual + stds_manual, color=MANUAL_COLOR, alpha=0.3)
ax_bias_std.set_xlabel("Coverage Angle [deg]")
ax_bias_std.set_ylabel("Error in DBH estimate [cm]")
fig_bias_std.savefig("/home/ori/Documents/Leonard/drs-docs-current/2024-iros-realtime-trees/pics/coverage_ablation_bias_std.pdf", bbox_inches='tight')